In [11]:

from pathlib import Path
import numpy as np
import random
from collections import Counter
from omnibelt import load_json


In [36]:

path = '../data/cladder-v1/cladder-v1-q-aggregate.json'
path = '../data/cladder-v1/cladder-v1-q-commonsense.json'
# path = '../data/cladder-v1-common-easy.json'
full = load_json(path)
full_ids = {entry['question_id']: entry for entry in full}
models = load_json('../data/cladder-v1/cladder-v1-meta-models.json')
model_table = {info['model_id']: info for info in models}
len(full), len(models)

(10392, 7064)

In [9]:
# models[1]

In [37]:
full[8003]

{'question_id': 21241,
 'desc_id': 'obesity_mortality-arrowhead-backadj-model4576-spec12-q1',
 'given_info': 'Method 1: We look directly at how obesity correlates with lifespan in general. Method 2: We look at this correlation case by case according to diabetes.',
 'question': 'To understand how obesity affects lifespan, is it more correct to use the Method 1 than Method 2?',
 'answer': 'yes',
 'meta': {'story_id': 'obesity_mortality',
  'graph_id': 'arrowhead',
  'flipped': False,
  'polarity': True,
  'groundtruth': [],
  'bad_candidate_set': ['V3'],
  'given_info': [[], ['V3']],
  'query_type': 'backadj',
  'rung': 2,
  'formal_form': '[backdoor adjustment set for Y given X]',
  'treatment': 'X',
  'outcome': 'Y',
  'model_id': 4576},
 'reasoning': None}

In [38]:
key = 'rung'
vocabs = {}
for entry in full:
	vocabs.setdefault(entry['meta'][key], []).extend(entry['question'].split())
	vocabs.setdefault(entry['meta'][key], []).extend(entry['given_info'].split())
	model = models[entry['meta']['model_id']]
	vocabs.setdefault(entry['meta'][key], []).extend(model['background'].split())
def _cleanup(w):
	w = w.lower()
	w = w.replace('?', '').replace('.', '').replace(',', '').replace('\'', '').replace('\"', '').replace('%', '').replace(':', '')
	# check if it's a number
	try:
		float(w)
		return 'NUM'
	except:
		pass
	return w
vocabs = {k: [_cleanup(w) for w in v] for k, v in vocabs.items()}
vocabs = {k: Counter(v) for k, v in vocabs.items()}
vocabs.keys()

dict_keys([2, 3, 1])

In [40]:
# vocabs

In [41]:
{k: len(v) for k, v in vocabs.items()}

{2: 314, 3: 387, 1: 315}

In [47]:
random.seed(0)
indices = random.choices(list(range(len(full))), k=50)
print(indices)
assert len(set(indices)) == len(indices)

[8775, 7876, 4370, 2690, 5313, 4208, 8145, 3152, 4952, 6062, 9437, 5244, 2928, 7854, 6426, 2603, 9454, 10213, 8419, 9375, 3223, 7584, 9340, 7107, 4906, 1046, 4511, 6348, 9488, 10044, 4957, 8992, 2707, 8365, 5702, 145, 7479, 4144, 8571, 6943, 11, 5129, 9016, 2534, 3379, 9045, 1985, 5897, 2479, 10054]


In [48]:
for i, ind in enumerate(indices):
	entry = full[ind]
	model = models[entry['meta']['model_id']]
	print(i)
	# print(model['background'])
	# print(entry['given_info'])
	# print(entry['question'])
	print(entry['answer'])
	print()

0
no

1
yes

2
no

3
yes

4
yes

5
no

6
no

7
no

8
yes

9
yes

10
no

11
no

12
yes

13
no

14
no

15
no

16
yes

17
yes

18
no

19
no

20
no

21
no

22
yes

23
yes

24
yes

25
no

26
yes

27
yes

28
no

29
no

30
yes

31
no

32
no

33
yes

34
yes

35
no

36
yes

37
no

38
yes

39
yes

40
no

41
yes

42
yes

43
no

44
yes

45
yes

46
yes

47
no

48
no

49
no



In [35]:
len([e for e in full if e['desc_id'].startswith('nonse')])/len(full)

0.7272727272727273